# What we want:

### A single number 1 -- 12 that describes the quality of a given observation.

This number should take into account:
1. Goodness of seeing, both from scintillation and from HSM/MFGS parameters
2. Duration of observations. Longer is better.
3. Interesting events "nearby"
4. Optionally: IRIS pointing in the same time span.

## The Quality is for the full observing *day*, not per pointing.

#### Breaking it down:
1. Goodness of seeing:
    a. Scrape the DCSS log or FIRS files for scintillation monitor values
    b. Read in the ROSA/Zyla seeing params, and determine the spread in these values
    c. Place these values in a csv file, with observation type.
2. Duration of observations.
    a. Read by start and end time in FITS files. The flow chart to check:
        i. ROSA
        ii. FIRS
        iii. SPINOR/HSG
        iv. NEVER Zyla
    b. Total obs time in the same csv file
3. Interesting events "nearby"
    a. Read in the pointing_info.txt file.
        i. If the number of pointings and starttimes is the same, use the obssum start/end times
    b. Query HEK for coronal rain, filament eruptions, and flares.
    c. Check for anything from the pointing center to a max radius of 200" (FOV is 180")
    d. If it's within 100", with a starttime that's between our start and end, add 2 to metric
    e. If it satisfies one condition but not the other, add 1
    f: If it is outside 100" and the starttime is within 30 minutes of our window, add 0.5

# Question

##### Is it worth going back to my obssum code, and modifying it to write the endtime as well? It would save some time....

###### I did it....

In [2]:
%config Completer.use_jedi = False

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
import glob
import tqdm

from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
from sunpy.net import Fido, attrs as a
import astropy.units as u

In [3]:
params = {
    'savefig.dpi':300,
    'axes.labelsize':12,
    'axes.labelweight':'bold',
    'axes.titleweight':'bold',
    'figure.titleweight':'bold',
    'axes.titlesize':14,
    'font.size':12,
    'legend.fontsize':12,
    'font.family':'serif'
}
plt.rcParams.update(params)

In [4]:
def quality_metaparams(indir, date):
    """Sets up information needed for our quality metrics.
    Outputs a structure with:
    OBSTYPE
    DURATION
    HSM Coefficient of Variation (GBAND)
    HSM Coefficient of Variation (4170)
    HSM Coefficient of Variation (CAK)
    MFGS Coefficient of Variation (ZYLA)
    (CV is stdev/mean)
    EVENT FACTOR
    EVENT FACTOR TYPE
    """
    #First, check if there's pointing info
    pinfo = os.path.join(os.path.join(indir, "context_ims"),"pointing_info.txt")
    if not os.path.isfile(pinfo):
        return None
    pointings = np.recfromtxt(pinfo, delimiter=',',names=True,encoding=None)
    pointings = np.atleast_1d(pointings)
    if len(pointings.TIME) == 0:
        return None
    # Cool. There's pointing info.
    # Now, check if there's seeing info.
    all_seeing = sorted(glob.glob(os.path.join(indir, "*seeing_quality.npy")))
    if len(all_seeing) == 0:
        return None
    # Cool. There's seeing info.
    
    # OBSTYPE
    obsreg = open("/var/www/html/observations_registry.csv","r")
    oreg_lines = obsreg.readlines()
    obsreg.close()
    all_datestrs = [l.split(",")[1] for l in oreg_lines]
    all_otypes = [l.split(",")[2] for l in oreg_lines]
    obstype = all_otypes[all_datestrs.index(date)]
    
    duration = np.timedelta64(0,"m")
    for i in range(len(pointings.TIME)):
        duration += np.datetime64(pointings.END[i]) - np.datetime64(pointings.TIME[i])
    
    zyla_cv = []
    gband_cv = []
    cak_cv = []
    cont_cv = []
    ibis_cv = []
    
    for i in range(len(all_seeing)):
        if "zyla" in all_seeing[i]:
            see = np.load(all_seeing[i])
            zyla_cv.append(np.nanstd(see['MFGS'])/np.nanmean(see['MFGS']))
        else:
            try:
                see = np.load(all_seeing[i])
            except:
                see = np.load(all_seeing[i],allow_pickle=True).flat[0]
            if 'gband' in all_seeing[i]:
                gband_cv.append(np.nanstd(see['HSM'])/np.nanmean(see['HSM']))
            elif '4170_' in all_seeing[i]:
                cont_cv.append(np.nanstd(see['HSM'])/np.nanmean(see['HSM']))
            elif 'cak_' in all_seeing[i]:
                cak_cv.append(np.nanstd(see['HSM'])/np.nanmean(see['HSM']))
            elif 'ibis' in all_seeing[i]:
                ibis_cv.append(np.nanstd(see['HSM'])/np.nanmean(see['HSM']))
                
    if len(zyla_cv) == 0:
        zyla_cv = np.nan
    else:
        zyla_cv = np.nanmean(np.array(zyla_cv))
    if len(gband_cv) == 0:
        gband_cv = np.nan
    else:
        gband_cv = np.nanmean(np.array(gband_cv))
    if len(cak_cv) == 0:
        cak_cv = np.nan
    else:
        cak_cv = np.nanmean(np.array(cak_cv))
    if len(cont_cv) == 0:
        cont_cv = np.nan
    else:
        cont_cv = np.nanmean(np.array(cont_cv))
    if len(ibis_cv) == 0:
        ibis_cv = np.nan
    else:
        ibis_cv = np.nanmean(np.array(cont_cv))
    # Cool. We've got duration and seeing. 
    # Now for the hard part.
    # Our fudge factor for events
    
    # Conceivably, there might be multiple flares/other events in frame.
    # In this case, we want the max fudge factor, so append to a list
    # NOTE: Coronal rain doesn't necessarily have an associated time.
    # For CR, if there's an event, it's close enough temporally by default.
    event_factors = []
    events = []
    see_cv = []
    for i in range(len(pointings.TIME)):
        tele_pointing = SkyCoord(
            pointings['SLON'][i]*u.deg,
            pointings['SLAT'][i]*u.deg,
            frame=frames.HeliographicStonyhurst,
            observer='earth',
            obstime=pointings['TIME'][i]
        )
        tele_pointing = tele_pointing.transform_to(frames.Helioprojective)
        tele_start = np.datetime64(pointings.TIME[i])
        tele_end = np.datetime64(pointings.END[i])

        #### Real quick, add in scmon values...
        see_cv.append(float(pointings.SSCIN[i])/float(pointings.MSCIN[i]))
        
        #### Flares first.
        #### ...Cause I care about them the most.
        flare_fudge = []
        flare_events = []
        flare_query = Fido.search(
            a.Time(
                (tele_start - np.timedelta64(30,'m')).astype(str),
                (tele_end + np.timedelta64(30,'m')).astype(str)
            ),
            a.hek.EventType('FL'),
            a.hek.FL.GOESCls>'B1.0'
        )
        if len(flare_query['hek']) > 0:
            flare_xcd = flare_query['hek']['event_coord1']
            flare_ycd = flare_query['hek']['event_coord2']
            flare_peak = flare_query['hek']['event_peaktime']
            flare_class = flare_query['hek']['fl_goescls']
            flare_coordsys = flare_query['hek']['event_coordunit']
        else:
            flare_xcd = []
            flare_ycd = []
            flare_peak = []
            flare_class = []
            flare_coordsys = []
        for j in range(len(flare_xcd)):
            if flare_coordsys[j] == 'degrees':
                unit=u.deg
                frm = frames.HeliographicStonyhurst
            elif flare_coordsys[j] == 'arcsec':
                unit=u.arcsec
                frm=frames.Helioprojective
            flare_coord = SkyCoord(
                flare_xcd[j]*unit,
                flare_ycd[j]*unit,
                obstime=flare_peak[j],
                observer='earth',
                frame=frm
            )
            flare_coord = flare_coord.transform_to(frames.Helioprojective)
            del_x = float((tele_pointing.Tx - flare_coord.Tx).value)
            del_y = float((tele_pointing.Ty - flare_coord.Ty).value)
            distance = np.sqrt(del_x**2 + del_y**2)
            # Check if it's nearby:
            if distance <= 200:
                event_time = np.datetime64(flare_peak[j].value[0])
                # Best Case. Perfectly in field and on time
                if (event_time < tele_end) & (event_time > tele_start) & (distance <= 100):
                    flare_fudge.append(2)
                    flare_events.append(flare_class[j])
                # Second best case. Slightly out of time, but perfectly in field.
                elif (distance <= 100):
                    flare_fudge.append(1)
                    flare_events.append(flare_class[j])
                # Second best case. Perfectly in time, slightly out of field.
                elif (event_time < tele_end) & (event_time > tele_start):
                    flare_fudge.append(1)
                    flare_events.append(flare_class[j])
                # We've already passively selected this. Just out of FOV, and not in time.
                else:
                    flare_fudge.append(0.5)
                    flare_events.append(flare_class[j])
        # Check our flares. If there are any, append them to the event list.
        if len(flare_fudge) > 0:
            event_factors.append(np.nanmax(flare_fudge))
            events.append(flare_events[flare_fudge.index(np.nanmax(flare_fudge))])
        
        #### Filament eruptions Second.
        #### ...Cause they're like flares.
        filament_fudge = []
        filament_query = Fido.search(
            a.Time(
                (tele_start - np.timedelta64(30,'m')).astype(str),
                (tele_end + np.timedelta64(30,'m')).astype(str)
            ),
            a.hek.EventType('FE')
        )
        if len(filament_query['hek']) > 0:
            filament_xcd = filament_query['hek']['event_coord1']
            filament_ycd = filament_query['hek']['event_coord2']
            filament_peak = filament_query['hek']['event_peaktime']
            filament_coordsys = filament_query['hek']['event_coordunit']
        else:
            filament_xcd = []
            filament_ycd = []
            filament_peak = []
            filament_coordsys = []
        for j in range(len(filament_xcd)):
            if filament_coordsys[j] == 'degrees':
                unit=u.deg
                frm = frames.HeliographicStonyhurst
            elif filament_coordsys[j] == 'arcsec':
                unit=u.arcsec
                frm=frames.Helioprojective
            filament_coord = SkyCoord(
                filament_xcd[j]*unit,
                filament_ycd[j]*unit,
                obstime=filament_peak[j],
                observer='earth',
                frame=frm
            )
            filament_coord = filament_coord.transform_to(frames.Helioprojective)
            del_x = float((tele_pointing.Tx - filament_coord.Tx).value)
            del_y = float((tele_pointing.Ty - filament_coord.Ty).value)
            distance = np.sqrt(del_x**2 + del_y**2)
            # Check if it's nearby:
            if distance <= 200:
                event_time = np.datetime64(filament_peak[j].value[0])
                # Best Case. Perfectly in field and on time
                if (event_time < tele_end) & (event_time > tele_start) & (distance <= 100):
                    filament_fudge.append(2)
                # Second best case. Slightly out of time, but perfectly in field.
                elif (distance <= 100):
                    filament_fudge.append(1)
                # Second best case. Perfectly in time, slightly out of field.
                elif (event_time < tele_end) & (event_time > tele_start):
                    filament_fudge.append(1)
                # We've already passively selected this. Just out of FOV, and not in time.
                else:
                    filament_fudge.append(0.5)
        # Check our filaments. If there are any, append them to the event list.
        if len(filament_fudge) > 0:
            event_factors.append(np.nanmax(filament_fudge))
            events.append('Filament Eruption')
            
    
        #### Coronal Rain last
        #### Cause it's calm.
        rain_fudge = []
        rain_query = Fido.search(
            a.Time(
                (tele_start - np.timedelta64(30,'m')).astype(str),
                (tele_end + np.timedelta64(30,'m')).astype(str)
            ),
            a.hek.EventType('CR')
        )
        if len(rain_query['hek']) > 0:
            rain_xcd = rain_query['hek']['event_coord1']
            rain_ycd = rain_query['hek']['event_coord2']
            rain_coordsys = rain_query['hek']['event_coordunit']
        else:
            rain_xcd = []
            rain_ycd = []
            rain_coordsys = []
        for j in range(len(rain_xcd)):
            if rain_coordsys[j] == 'degrees':
                unit=u.deg
                frm = frames.HeliographicStonyhurst
            elif rain_coordsys[j] == 'arcsec':
                unit=u.arcsec
                frm=frames.Helioprojective
            rain_coord = SkyCoord(
                rain_xcd[j]*unit,
                rain_ycd[j]*unit,
                obstime=tele_start.astype(str),
                observer='earth',
                frame=frm
            )
            rain_coord = rain_coord.transform_to(frames.Helioprojective)
            del_x = float((tele_pointing.Tx - rain_coord.Tx).value)
            del_y = float((tele_pointing.Ty - rain_coord.Ty).value)
            distance = np.sqrt(del_x**2 + del_y**2)
            # Check if it's nearby:
            if distance <= 200:
                event_time = np.datetime64(rain_peak[j].value[0])
                # Best Case. Perfectly in field and on time
                if (distance <= 100):
                    rain_fudge.append(2)
                # Second best case. Slightly out of time, but perfectly in field.
                else:
                    rain_fudge.append(1)
        # Check our rains. If there are any, append them to the event list.
        if len(rain_fudge) > 0:
            event_factors.append(np.nanmax(rain_fudge))
            events.append('Coronal Rain')
    
    if len(event_factors) > 0:
        event_correction = np.nanmax(event_factors)
        event_correction_type = events[event_factors.index(np.nanmax(event_factors))]
    else:
        event_correction = 0
        event_correction_type = 'None'
        
    #### Putting it all together...
    if len(see_cv) == 0:
        see_cv = np.nan
    else:
        see_cv = np.nanmean(np.array(see_cv))
    
    data_quality = {
        'OBSTYPE':obstype,
        'DURATION':duration.astype('timedelta64[m]').astype(int),
        'ZYLA_CV':zyla_cv,
        'CONT_CV':cont_cv,
        'GBAND_CV':gband_cv,
        'CAK_CV':cak_cv,
        'IBIS_CV':ibis_cv,
        'SEE_CV':see_cv,
        'EVENT_FACTOR':event_correction,
        'EVENT_TYPE':event_correction_type
    }
    return data_quality

In [14]:
all_dirs = sorted(glob.glob("/sunspot/solardata/20*/*/*/"))[1:]
all_dates = [d.split("/")[-4] + '-' + d.split("/")[-3] + '-' + d.split("/")[-2] for d in all_dirs]

master_file = open("quality_params.txt","w")
master_file.write("DATE,OBSTYPE,DURATION,ZYLA_CV,CONT_CV,GBAND_CV,CAK_CV,IBIS_CV,SEE_CV,EVENT_FACTOR,EVENT_TYPE\n")
for i in tqdm.tqdm(range(len(all_dirs))):
    try:
        dq_dict = quality_metaparams(all_dirs[i], all_dates[i])
    except:
        dq_dict = None
    if dq_dict:
        write_list = [all_dates[i]]
        for key in list(dq_dict.keys()):
            write_list.append(str(dq_dict[key]))
        write_str = ','.join(write_list)
        master_file.write(
            write_str + "\n"
        )
    else:
        write_str = all_dates[i]+',nan,nan,nan,nan,nan,nan,nan,nan,nan,nan\n'
        master_file.write(write_str)
master_file.close()

  3%|█▍                                        | 14/417 [00:25<17:08,  2.55s/it]/home/sellers/.conda/envs/ssoc/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_11584/4105711041.py:50: RuntimeWarning: Mean of empty slice
  zyla_cv.append(np.nanstd(see['MFGS'])/np.nanmean(see['MFGS']))
/tmp/ipykernel_11584/4105711041.py:68: RuntimeWarning: Mean of empty slice
  zyla_cv = np.nanmean(np.array(zyla_cv))
  5%|██                                        | 20/417 [00:57<33:50,  5.11s/it]/tmp/ipykernel_11584/4105711041.py:84: RuntimeWarning: Mean of empty slice
  ibis_cv = np.nanmean(np.array(cont_cv))
  5%|██▏                                       | 22/417 [01:07<33:09,  5.04s/it]/tmp/ipykernel_11584/4105711041.py:84: RuntimeWarning: Mean of empty slice
  ibis_cv = np.nanmean(np.array(cont_cv))
 12%|████▉                                     | 49/417 [02:57<29

In [5]:
qp = np.genfromtxt("quality_control/quality_params.txt",
                  delimiter=',',
                  names=True,
                  encoding=None,
                  dtype=None)

In [6]:
qp.dtype.names

('DATE',
 'OBSTYPE',
 'DURATION',
 'ZYLA_CV',
 'CONT_CV',
 'GBAND_CV',
 'CAK_CV',
 'IBIS_CV',
 'SEE_CV',
 'EVENT_FACTOR',
 'EVENT_TYPE')

In [98]:
pctile_range = np.arange(0,100,1)

In [99]:
zyla_cv = qp['ZYLA_CV'][qp['OBSTYPE'] == 'flare']
zyla_cv = zyla_cv[~np.isnan(zyla_cv)]

dur = qp['DURATION'][qp['OBSTYPE']=='flare']
dur = dur[~np.isnan(dur)]

In [100]:
np.percentile(dur, pctile_range)

array([ 12.  ,  18.9 ,  23.9 ,  27.21,  29.28,  30.9 ,  32.28,  33.66,
        37.64,  41.  ,  41.  ,  43.95,  46.84,  48.91,  50.32,  51.  ,
        51.  ,  51.  ,  51.  ,  51.  ,  51.  ,  51.  ,  51.  ,  51.  ,
        51.  ,  51.  ,  51.  ,  51.  ,  51.  ,  51.01,  51.7 ,  52.39,
        53.08,  53.77,  54.46,  55.6 ,  58.36,  60.06,  61.  ,  61.  ,
        61.  ,  61.  ,  61.  ,  61.  ,  61.  ,  61.05,  61.74,  62.  ,
        62.12,  62.81,  63.  ,  63.19,  63.88,  64.  ,  64.26,  64.95,
        65.  ,  65.99,  68.04,  69.42,  70.8 ,  72.63,  77.46,  79.94,
        81.16,  81.85,  82.  ,  82.  ,  82.  ,  82.  ,  82.  ,  82.  ,
        82.  ,  85.33,  91.06,  91.75,  92.  ,  92.39,  94.46,  97.04,
        99.8 , 102.56, 103.  , 103.27, 103.96, 105.95, 109.04, 113.  ,
       113.  , 113.82, 115.  , 115.  , 120.76, 127.51, 129.58, 130.  ,
       131.44, 135.58, 154.6 , 168.48])

In [83]:
np.argwhere(45 >= np.percentile(dur, pctile_range)).max()

2

In [47]:
zyla_cv.min()

0.01845251903136418

In [89]:
percentiles = np.percentile(zyla_cv, pctile_range)

In [90]:
percentiles

array([0.01845252, 0.02533617, 0.02929027, 0.03124474, 0.03480364,
       0.03776052, 0.03873575, 0.0428634 , 0.04479267, 0.04672167,
       0.04732347, 0.05217127, 0.05341476, 0.05708229, 0.05767232,
       0.05978218, 0.06739226, 0.06767359, 0.07116653, 0.07584223])

In [93]:
np.argwhere(0.041 >= percentiles).max()

6

In [58]:
percentiles[8]

0.04479267331673768

In [59]:
zyla_cv[50]

0.04396471206465507

In [34]:
len(percentiles)

20

In [78]:
100-pctile_range[1]

95

## See above for how to get percentile values to 5% level

Now we'll apply this to the whole archive, writing the mean seeing percentile and duration percentile. We'll do this individually for every date in the flare, filament, psp, and qs data sets, and for the whole archive. Each file will have the date, the mean seeing percentile, the duration percentile, and the event factors. That'll give us everything we need for putting the numbers on the webpages.

In [102]:
flare_quality = open("quality_control/flare_quality.txt","w")
filament_quality = open("quality_control/filament_quality.txt","w")
psp_quality = open("quality_control/psp_quality.txt","w")
qs_quality = open("quality_control/qs_quality.txt","w")
archive_quality = open("quality_control/archive_quality.txt","w")

header = 'DATE,OBSTYPE,QUALITY,SEEING_FACTOR,DURATION_FACTOR,EVENT_FACTOR,EVENT_TYPE\n'
flare_quality.write(header)
filament_quality.write(header)
psp_quality.write(header)
qs_quality.write(header)
archive_quality.write(header)

dur_keys = ['ZYLA_CV','CONT_CV','GBAND_CV','CAK_CV','IBIS_CV','SEE_CV']
obstypes = ['flare','filament','qs','psp']
otype_files = [flare_quality, filament_quality, qs_quality, psp_quality]

for i in tqdm.tqdm(range(len(qp['DATE']))):
    if qp['OBSTYPE'][i] != b'nan':
        # Full-archive quality first
        seeing_percentile = []
        for key in dur_keys:
            if not np.isnan(qp[key][i]):
                full_cv = qp[key][~np.isnan(qp[key])]
                percentiles = np.percentile(full_cv, pctile_range)
                seeing_percentile.append(
                    100 - pctile_range[
                        np.argwhere(qp[key][i] >= percentiles).max()
                    ]
                )
        if len(seeing_percentile) > 0:
            seeing_factor = round(np.nanmean(seeing_percentile)/10,1)
        else:
            seeing_factor = np.nan
        duration_percentile = []
        if not np.isnan(qp['DURATION'][i]):
            full_dur = qp['DURATION'][~np.isnan(qp['DURATION'])]
            percentiles = np.percentile(full_dur, pctile_range)
            duration_percentile.append(
                pctile_range[
                    np.argwhere(qp['DURATION'][i] >= percentiles).max()
                ]
            )
        if len(duration_percentile) > 0:
            duration_factor = duration_percentile[0]/10
        else:
            duration_factor = np.nan
        if (not np.isnan(seeing_factor)) & (not np.isnan(duration_factor)):
            quality = round(np.nanmean([seeing_factor,duration_factor]) + qp['EVENT_FACTOR'][i],1)
        else:
            quality = np.nan
        write_list = [
            qp['DATE'][i],
            qp['OBSTYPE'][i],
            str(quality),
            str(seeing_factor),
            str(duration_factor),
            str(qp['EVENT_FACTOR'][i]),
            qp['EVENT_TYPE'][i]
        ]
        write_str = ','.join(write_list) + "\n"
        archive_quality.write(write_str)
        
        # Now we do it all the obstypes...
        for j in range(len(obstypes)):
            if qp['OBSTYPE'][i] == obstypes[j]:
                seeing_percentile = []
                for key in dur_keys:
                    if not np.isnan(qp[key][i]):
                        full_cv = qp[key][qp['OBSTYPE'] == obstypes[j]]
                        full_cv = full_cv[~np.isnan(full_cv)]
                        percentiles = np.percentile(full_cv, pctile_range)
                        seeing_percentile.append(
                            100 - pctile_range[
                                np.argwhere(qp[key][i] >= percentiles).max()
                            ]
                        )
                if len(seeing_percentile) > 0:
                    seeing_factor = round(np.nanmean(seeing_percentile)/10,1)
                else:
                    seeing_factor = np.nan
                duration_percentile = []
                if not np.isnan(qp['DURATION'][i]):
                    full_dur = qp['DURATION'][qp['OBSTYPE'] == obstypes[j]]
                    full_dur = full_dur[~np.isnan(full_dur)]
                    percentiles = np.percentile(full_dur, pctile_range)
                    duration_percentile.append(
                        pctile_range[
                            np.argwhere(qp['DURATION'][i] >= percentiles).max()
                        ]
                    )
                if len(duration_percentile) > 0:
                    duration_factor = duration_percentile[0]/10
                else:
                    duration_factor = np.nan
                if (not np.isnan(seeing_factor)) & (not np.isnan(duration_factor)):
                    quality = round(np.nanmean([seeing_factor,duration_factor]) + qp['EVENT_FACTOR'][i], 2)
                else:
                    quality = np.nan
                write_list = [
                    qp['DATE'][i],
                    qp['OBSTYPE'][i],
                    str(quality),
                    str(seeing_factor),
                    str(duration_factor),
                    str(qp['EVENT_FACTOR'][i]),
                    qp['EVENT_TYPE'][i]
                ]
                write_str = ','.join(write_list) + "\n"
                otype_files[j].write(write_str)
                
for file in otype_files:
    file.close()
archive_quality.close()
        

100%|████████████████████████████████████████| 417/417 [00:00<00:00, 733.14it/s]


In [87]:
percentiles

array([6.86289350e-05, 5.43909170e-04, 7.42749427e-04, 9.64981600e-04,
       1.26283296e-03, 1.52657664e-03, 2.08916300e-03, 2.38100446e-03,
       2.61023097e-03, 3.08866956e-03, 3.41000345e-03, 4.20144756e-03,
       4.67762175e-03, 5.12932094e-03, 5.54840034e-03, 6.48989595e-03,
       6.89874773e-03, 7.27481020e-03, 8.21661970e-03, 1.02483302e-02])

In [86]:
qp[key][i]

0.015681340392054124